In [1]:
import torch
import scipy as sp
import scipy.misc
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import imageio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms 
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
from torch.autograd import Variable
from PIL import Image
from PIL import ImageChops
from PIL import ImageEnhance
import torch.optim as optim
import tqdm
import copy
import utils
import nyu_dataset
import pix2pix_model
import tqdm

In [2]:
num_classes = 40

In [3]:
segmentation_dataset_train = nyu_dataset.SegmentationDataset(transforms=nyu_dataset.SegmentationTransform())
data_loader_train = DataLoader(segmentation_dataset_train, batch_size=8, shuffle=True, num_workers=1)

segmentation_dataset_validation = nyu_dataset.SegmentationDataset(path_to_datafolder='./datasets/nyu/val/', 
                                                                  transforms=
                                                                  nyu_dataset.SegmentationTransform(False))
data_loader_val = DataLoader(segmentation_dataset_validation, batch_size=1, shuffle=False, num_workers=1)

In [4]:
generator = pix2pix_model.Generator(num_classes, 3).cuda(1)
discriminator = pix2pix_model.Discriminator(num_classes + 3).cuda(1)

In [ ]:
def train(data_loader_train, data_loader_val, generator, discriminator, num_epoch=100, lambda_param=10.0):
    d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    g_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    
    GANLoss = nn.BCELoss()
    L1Loss = nn.L1Loss()
    
    for epoch in tqdm.tqdm(range(num_epoch)):
        for image_batch, segmentation_batch in data_loader_train:
            image_batch = Variable(image_batch.cuda(1), requires_grad=False)
            segmentation_batch = Variable(segmentation_batch.cuda(1), requires_grad=False)
            
            fake_image_batch = generator.forward(segmentation_batch)
            
            # discriminator step
            d_optimizer.zero_grad()
            
            fake_input_discriminator = torch.cat((segmentation_batch, fake_image_batch), 1)
            real_input_discriminator = torch.cat((segmentation_batch, image_batch), 1)
            
            predictions_fake = discriminator.forward(fake_input_discriminator.detach())
            predictions_real = discriminator.forward(real_input_discriminator)
            
            target_tensor = Variable(torch.ones(predictions_fake.shape).cuda(1), requires_grad=False)
            loss_discriminator_real = GANLoss(predictions_real, target_tensor)
            
            target_tensor.data.fill_(0)
            loss_discriminator_fake = GANLoss(predictions_fake, target_tensor)
            
            loss_discriminator = (loss_discriminator_fake + loss_discriminator_real)*0.5
            loss_discriminator.backward()
            d_optimizer.step()
            
            # generator step
            g_optimizer.zero_grad()
            
            fake_input_discriminator = torch.cat((segmentation_batch, fake_image_batch), 1)         
            predictions_fake = discriminator.forward(fake_input_discriminator)
            
            target_tensor.data.fill_(1)
            loss_generator_gan = GANLoss(predictions_fake, target_tensor)
            
            loss_generator_L1 = L1Loss(fake_image_batch, image_batch)*lambda_param
            
            loss_generator = loss_generator_L1 + loss_generator_gan
            loss_generator.backward()
            g_optimizer.step()
        
        for image, segmentation in data_loader_val:
            segmentation = Variable(segmentation.cuda(1), requires_grad=False)   
            generated_image = generator.forward(segmentation)
            
            print('Val on epoch = {}'.format(epoch))
            plt.imshow(np.rollaxis(generated_image.cpu().data.numpy()[0], 0, 3))
            plt.show()
            break

In [ ]:
train(data_loader_train, data_loader_val, generator, discriminator, num_epoch=1)

  0%|          | 0/1 [00:00<?, ?it/s]